In [ ]:
!git clone https://github.com/TusharW4ni/AirWriteML.git

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input
import matplotlib.pyplot as plt

def load(file_path):
    df = pd.read_csv(file_path)
    X = df[['index', 'x', 'y', 'z']].values
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    return X

letter_samples = {
    'A': [load(f'./AirWriteML/data/A/A_sample_{i}.csv') for i in range(1, 51)],
    'B': [load(f'./AirWriteML/data/B/B_sample_{i}.csv') for i in range(1, 51)],
    'C': [load(f'./AirWriteML/data/C/C_sample_{i}.csv') for i in range(1, 51)],
    'D': [load(f'./AirWriteML/data/D/D_sample_{i}.csv') for i in range(1, 51)],
    'E': [load(f'./AirWriteML/data/E/E_sample_{i}.csv') for i in range(1, 51)],
    'H': [load(f'./AirWriteML/data/H/H_sample_{i}.csv') for i in range(1, 51)],
    'I': [load(f'./AirWriteML/data/I/I_sample_{i}.csv') for i in range(1, 51)],
    'L': [load(f'./AirWriteML/data/L/L_sample_{i}.csv') for i in range(1, 51)],
    'N': [load(f'./AirWriteML/data/N/N_sample_{i}.csv') for i in range(1, 51)],
    'O': [load(f'./AirWriteML/data/O/O_sample_{i}.csv') for i in range(1, 51)],
    'R': [load(f'./AirWriteML/data/R/R_sample_{i}.csv') for i in range(1, 51)],
    'S': [load(f'./AirWriteML/data/S/S_sample_{i}.csv') for i in range(1, 51)],
    'T': [load(f'./AirWriteML/data/T/T_sample_{i}.csv') for i in range(1, 51)],
}

max_length = max(len(sample) for samples in letter_samples.values() for sample in samples)

X = []
y = []
for label, samples in letter_samples.items():
    for sample in samples:
        padded_sample = np.pad(sample, ((0, max_length - len(sample)), (0, 0)), 'constant')
        X.append(padded_sample)
        y.append(label)

X = np.array(X)
y = np.array(y)

label_to_index = {label: index for index, label in enumerate(sorted(set(y)))}
y_numeric = np.array([label_to_index[label] for label in y])

y_onehot = to_categorical(y_numeric)

In [ ]:
def create_model(input_shape, num_classes):
  model = Sequential([
      Input(shape=input_shape),
      LSTM(64, return_sequences=True),
      LSTM(32),
      Dense(num_classes, activation='softmax')
  ])
  model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
num_classes = len(label_to_index)

fold_no = 1
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_onehot[train_index], y_onehot[test_index]
    print(X.shape[1])
    print(X.shape[2])
    model = create_model((X.shape[1], X.shape[2]), num_classes)

    history = model.fit(X_train, y_train,
                        batch_size=32,
                        epochs=200,
                        validation_data=(X_test, y_test),
                        verbose=1)

    # Plotting the graphs for each fold
    plt.figure(figsize=(14, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Fold {fold_no} - Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {fold_no} - Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

    fold_no += 1

model.save('air_write_model.h5')
import json
with open('label_to_index.json', 'w') as f:
    json.dump(label_to_index, f)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_true_classes, y_pred_classes, target_names=sorted(label_to_index.keys())))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=sorted(label_to_index.keys()), yticklabels=sorted(label_to_index.keys()))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

---
---
---



In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

def create_model(input_shape, num_classes):
  model = Sequential([
      Input(shape=input_shape),
      LSTM(64, return_sequences=True),
      LSTM(32),
      Dense(num_classes, activation='softmax')
  ])
  model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def load_and_preprocess(file_path):
    df = pd.read_csv(file_path)
    X = df[['index', 'x', 'y', 'z']].values
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    return X

def predict_letter(model, file_path, max_length, label_to_index):
    X = load_and_preprocess(file_path)

    if len(X) < max_length:
        padded_sample = np.pad(X, ((0, max_length - len(X)), (0, 0)), 'constant')
    else:
        padded_sample = X[:max_length]

    padded_sample = np.expand_dims(padded_sample, axis=0)

    pred = model.predict(padded_sample)
    pred_class = np.argmax(pred, axis=1)[0]

    return list(label_to_index.keys())[pred_class]

input_shape = (max_length, X.shape[2])
num_classes = len(label_to_index)
model = create_model(input_shape, num_classes)
model.load_weights('air_write_model.h5')

file_path = 'file.csv'
predicted_letter = predict_letter(model, file_path, max_length, label_to_index)

print(f"Predicted Letter: {predicted_letter}")
